In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from tqdm import tqdm
import os
import ray
from itertools import combinations
from collections import Counter

In [27]:
ray.init(num_cpus=16, ignore_reinit_error=True)

2024-07-25 20:08:28,690	INFO worker.py:1788 -- Started a local Ray instance.


Python version:,3.9.19
Ray version:,2.32.0


In [3]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [248]:
def extract_keywords_tfidf(text, num_keywords=20):
    okt = Okt()

    # TF-IDF 벡터화기 생성
    vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=1)
    X = vectorizer.fit_transform([text])  # 문자열 전체를 하나의 문서로 처리

    # 단어와 TF-IDF 점수 매핑
    tfidf_scores = X.sum(axis=0).A1
    words = vectorizer.get_feature_names_out()

    # 단어와 TF-IDF 점수를 데이터프레임으로 변환
    tfidf_df = pd.DataFrame({'Word': words, 'TF-IDF Score': tfidf_scores})

    # TF-IDF 점수로 정렬하여 상위 num개 키워드 출력
    return tfidf_df.sort_values(by='TF-IDF Score', ascending=False).head(num_keywords)

In [5]:
def extract_keywords_textrank(text, num_keywords=20):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    print(word_counts)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    #return top_keywords

    return [keyword for keyword, _ in top_keywords]

In [6]:
def extract_keywords_df(df, num=20): #func=extract_keywords_tfidf, num=20):
    doc_keywords = []
    
    for doc in df['text'] :
        #keywords = extract_keywords_tfidf(doc, num)
        #doc_keywords.append(keywords)
        doc_keywords.append(doc[:10])
        
    #df.loc[:, 'keywords'] = doc_keywords
    
    return doc_keywords

In [7]:
def test(text):
    
    
    
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    return text[:20]

In [242]:
@ray.remote
# 각 데이터 프레임 청크에서 키워드 추출 함수 적용
def process_chunk(chunk):
    
    def tokenize(text):
        # 형태소 분석을 통해 명사만 추출
        return [word for word in Okt().nouns(text) if len(word) > 1]
    
    def test(text):
                
        # 형태소 분석 및 명사 추출
        words = tokenize(text) 
        print(type(words))
        
        return words #text[:10]
    
    def extract_keywords_textrank(text, num_keywords=20):
   
        # 형태소 분석 및 명사 추출
        words = tokenize(text)
    
        # 단어들의 출현 빈도수 계산
        word_counts = Counter(words)

        # 그래프 생성
        graph = nx.Graph()

        # 단어 노드 추가
        for word, count in word_counts.items():
            if count > 1:  # 최소 출현 빈도 조건
                graph.add_node(word, count=count)

        # 단어의 연결을 위한 윈도우 크기 설정
        window_size = 4

        # 윈도우 안에서 단어 간의 연결 설정
        for i in range(len(words) - window_size + 1):
            window_words = words[i:i + window_size]
            for w1, w2 in combinations(window_words, 2):
                if graph.has_node(w1) and graph.has_node(w2):
                    if graph.has_edge(w1, w2):
                        graph[w1][w2]['weight'] += 1
                    else:
                        graph.add_edge(w1, w2, weight=1)

        # PageRank 계산
        rank = nx.pagerank(graph, weight='weight')

        # 상위 num_keywords개의 키워드 추출
        top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]

        return top_keywords

        #return [keyword for keyword, _ in top_keywords]
        
    def extract_keyworks_tfidf(text, num_keywords=20):
        # TF-IDF 벡터화기 생성
        vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=1)
        X = vectorizer.fit_transform([text])  # 문자열 전체를 하나의 문서로 처리

        # 단어와 TF-IDF 점수 매핑
        tfidf_scores = X.sum(axis=0).A1
        words = vectorizer.get_feature_names_out()

        # 단어와 TF-IDF 점수를 데이터프레임으로 변환
        tfidf_df = pd.DataFrame({'Word': words, 'TF-IDF Score': tfidf_scores})

        # TF-IDF 점수로 정렬하여 상위 num개 키워드 출력
        return tfidf_df.sort_values(by='TF-IDF Score', ascending=False).head(num_keywords)

    chunk['keywords'] = chunk['text'].apply(extract_keywords_textrank)

    return chunk

In [9]:
# 데이터 프레임을 청크로 나누는 함수
def split_dataframe(df, chunk_size):
    chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
    
    return chunks

In [69]:
def parse_text_file(file_path):
    result_dict = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄의 앞뒤 공백 제거
            line = line.strip()

            # C?? 형식의 코드와 나머지 문자열 분리
            if line:
                parts = line.split(' ', 1)  # 첫 번째 공백으로 나눔
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(',')  # 쉼표와 공백으로 구분하여 리스트 생성
                    result_dict[key] = values

    return result_dict

In [143]:
def find_matches(keyword_list, reference_list):
    matches = []
    
    for t in reference_list:
        if t[0] in keyword_list :
            matches.append(t[0])
            
    return matches

In [142]:
def count_matches(keyword_list, reference_list):
    count = 0
    for t in reference_list:
        if t[0] in keyword_list :
            count += 1
            
    return count

In [350]:
def get_keyword_only(keyword_pairs):
    keyword_list = []
    
    for p in keyword_pairs:
        keyword_list.append(p[0])
        
    return keyword_list

In [10]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [40]:
len(df)

3611424

In [30]:
len(df) / 2

1805712.0

In [41]:
length = len(df) #int(len(df) / 2)

In [42]:
length

3611424

In [43]:
length / 16

225714.0

In [44]:
len(df[:length])

3611424

In [45]:
chunk_size = int(length/16)
chunks = split_dataframe(df, chunk_size)

In [46]:
len(chunks[0])

225714

In [47]:
# Ray를 사용하여 병렬 처리
futures = [process_chunk.remote(chunk) for chunk in chunks]

# tqdm을 사용하여 진행 상황 표시
results = []
for future in tqdm(ray.get(futures), total=len(futures), desc="Processing"):
    results.append(future)

#result_chunks = ray.get(futures)

(raylet) Spilled 24532 MiB, 52 objects, write throughput 188 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
Processing: 100%|███████████████████████████| 16/16 [00:00<00:00, 309257.44it/s]


In [57]:
combined_df = pd.concat(results, ignore_index=True)

In [62]:
df_textrank = combined_df

In [65]:
df_textrank

,id,topic,text,keywords
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643..."
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231..."
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402..."
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118..."
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118..."
...,...,...,...,...
3611419,NZRW2200000001.2264,사회,日기자까지 통신조회 공수처에 아사히 “경위 밝히라” 일본 아사히신문이 30일 자사 ...,"[(기자, 0.07364789226727272), (조회, 0.05843554535..."
3611420,NZRW2200000001.2265,사회,오늘 ‘노동자’로 인정받고 내일 쫓겨나는 방송작가 “이제 MBC ‘뉴스외전’ 작가는...,"[(작가, 0.05351025601963465), (근로, 0.05099118966..."
3611421,NZRW2200000001.2266,사회,“내가 잣 도둑?” 앙심 품고 기자에 칼 겨눈 50대 집행유예 자신을 ‘잣 도둑’이...,"[(기자, 0.08305686810783024), (재판, 0.04992066025..."
3611422,NZRW2200000001.2269,사회,지상파 최초 KBS ‘비정규직 관리시스템’ 추진 KBS가 2023년 적용을 목표로 ...,"[(관리, 0.09326542432067701), (비정규직, 0.076670811..."


In [67]:
type(df_textrank.iloc[0].keywords[0])

tuple

In [59]:
combined_df.to_csv(directory_path+'/combined_news_keywords.tsv', sep='\t')

In [61]:
combined_df.iloc[0].keywords

[('가락시장', 0.03646997254429091),
 ('사람', 0.03454764351556988),
 ('수산시장', 0.03405599497941906),
 ('새벽', 0.03111153536702474),
 ('시장', 0.029838413039404724),
 ('배추', 0.028920909699708394),
 ('서민', 0.028031749849944568),
 ('경제', 0.02661029094416228),
 ('상인', 0.025143270050378596),
 ('가명', 0.024605998059264184),
 ('채소', 0.023819859082242937),
 ('도매', 0.02289853855588062),
 ('수산물', 0.022429575180303484),
 ('택시', 0.019913574914715182),
 ('중간', 0.019386854184506443),
 ('달라', 0.017582973506089887),
 ('작년', 0.017411372076471376),
 ('가격', 0.01630961155403351),
 ('하루', 0.01620970435601311),
 ('생선', 0.01618935790918644)]

In [736]:
file_path = '/home/osung/data/korean/KSIC_keyword.txt'

In [737]:
KSIC_key = parse_text_file(file_path)

In [738]:
KSIC_key

{'c101': ['도축장',
  '도축',
  '축산물',
  '도축업',
  '도축',
  '육류',
  '한우',
  '냉동',
  '토종닭',
  '축산',
  '마리',
  '사육',
  '고래고기',
  '햄',
  '소시지',
  '선지',
  '소고기',
  '돼지고기',
  '양고기',
  '오리고기',
  '닭고기',
  '훈제',
  '닭가슴살',
  '닭발',
  '닭갈비',
  '닭꼬치',
  '양꼬치',
  '돈까스',
  '돈가스',
  '족발',
  '갈비',
  '베이컨',
  '편육',
  '생산량',
  '쇠고기',
  '농민',
  '농가',
  '고기',
  '광우병',
  '귀농',
  '농촌',
  '돼지',
  '한돈',
  '불고기'],
 'c102': ['수산물',
  '양식',
  '수산',
  '광어',
  '항만',
  '해조',
  '어란',
  '어육',
  '캐비어',
  '어묵',
  '젓갈',
  '생선통조림',
  '생선페이스트',
  '건어물',
  '오징어',
  '우럭',
  '도미',
  '도다리',
  '가자미',
  '갈치',
  '고등어',
  '멍게',
  '해삼',
  '굴',
  '석화',
  '조개',
  '가리비',
  '키조개',
  '참돔',
  '돌돔',
  '줄돔',
  '방어',
  '쥐포',
  '먹태',
  '황태',
  '게장',
  '맛살',
  '생선가스',
  '생선까스',
  '액젓',
  '북어채',
  '황태채',
  '미역',
  '김부각',
  '구운김',
  '조미김',
  '생선',
  '김',
  '수확',
  '생산량',
  '어장',
  '어민',
  '아구',
  '아귀',
  '어선',
  '어업',
  '바지락',
  '어촌',
  '어업인',
  '멸치액젓',
  '멸치',
  '주꾸미',
  '어획량',
  '남해안',
  '서해안',
  '동해안',
  '수중',
  '백합',
  '대합',
  '생합'],
 'c103': ['농

In [739]:
(KSIC_key['c102'])

['수산물',
 '양식',
 '수산',
 '광어',
 '항만',
 '해조',
 '어란',
 '어육',
 '캐비어',
 '어묵',
 '젓갈',
 '생선통조림',
 '생선페이스트',
 '건어물',
 '오징어',
 '우럭',
 '도미',
 '도다리',
 '가자미',
 '갈치',
 '고등어',
 '멍게',
 '해삼',
 '굴',
 '석화',
 '조개',
 '가리비',
 '키조개',
 '참돔',
 '돌돔',
 '줄돔',
 '방어',
 '쥐포',
 '먹태',
 '황태',
 '게장',
 '맛살',
 '생선가스',
 '생선까스',
 '액젓',
 '북어채',
 '황태채',
 '미역',
 '김부각',
 '구운김',
 '조미김',
 '생선',
 '김',
 '수확',
 '생산량',
 '어장',
 '어민',
 '아구',
 '아귀',
 '어선',
 '어업',
 '바지락',
 '어촌',
 '어업인',
 '멸치액젓',
 '멸치',
 '주꾸미',
 '어획량',
 '남해안',
 '서해안',
 '동해안',
 '수중',
 '백합',
 '대합',
 '생합']

In [740]:
combined_key = []

In [741]:
for key in KSIC_key :
    combined_key.extend(KSIC_key[key])

In [742]:
len(combined_key)

2207

In [743]:
unique_keys = list(set(combined_key))

In [744]:
len(unique_keys)

2010

In [745]:
combined_df

,id,topic,text,keywords,match_keywords,match_count,keyword_only
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643...","[배추, 채소, 수산물, 생선]",4,"[가락시장, 사람, 수산시장, 새벽, 시장, 배추, 서민, 경제, 상인, 가명, 채..."
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231...",[],0,"[의원, 최소한, 망언, 국민, 인간, 우리, 품위, 대한, 국회의원, 라면, 문제..."
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402...",[운동],1,"[시민, 경제, 일찍, 공원, 기념, 운동, 보상, 정치인, 대구, 국채, 행사, ..."
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118...",[],0,"[시민, 대전, 촛불, 이명박, 언론, 악법, 새해, 촛불집회, 노래, 노조, 거리..."
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118...","[어민, 양식, 어장, 생산량, 수확, 농약]",6,"[어민, 생산, 장흥, 양식, 김양, 무산, 잡태, 처음, 어장, 생산량, 소비자,..."
...,...,...,...,...,...,...,...
3611419,NZRW2200000001.2264,사회,日기자까지 통신조회 공수처에 아사히 “경위 밝히라” 일본 아사히신문이 30일 자사 ...,"[(기자, 0.07364789226727272), (조회, 0.05843554535...",[신문],1,"[기자, 조회, 통신, 수사, 처가, 수집, 정보, 공수, 자료, 신문, 한국, 언..."
3611420,NZRW2200000001.2265,사회,오늘 ‘노동자’로 인정받고 내일 쫓겨나는 방송작가 “이제 MBC ‘뉴스외전’ 작가는...,"[(작가, 0.05351025601963465), (근로, 0.05099118966...",[],0,"[작가, 근로, 계약, 방송작가, 방송사, 노동부, 감독, 노동자, 시정, 인정, ..."
3611421,NZRW2200000001.2266,사회,“내가 잣 도둑?” 앙심 품고 기자에 칼 겨눈 50대 집행유예 자신을 ‘잣 도둑’이...,"[(기자, 0.08305686810783024), (재판, 0.04992066025...",[식칼],1,"[기자, 재판, 처벌, 판사, 피고인, 집행유예, 식칼, 폭행, 자신, 보도, 도둑..."
3611422,NZRW2200000001.2269,사회,지상파 최초 KBS ‘비정규직 관리시스템’ 추진 KBS가 2023년 적용을 목표로 ...,"[(관리, 0.09326542432067701), (비정규직, 0.076670811...",[],0,"[관리, 비정규직, 시스템, 문제, 인력, 처우, 실장, 직접, 개선, 부분, 계획..."


In [263]:
test_df = combined_df[:10]

In [264]:
test_df

,id,topic,text,keywords,match_keywords,match_count
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643...","[배추, 채소, 수산물, 생선]",4
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231...",[],0
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402...",[운동],1
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118...",[],0
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118...","[양식, 어장, 생산량, 수확, 농약]",5
5,NIRW1900000001.8,사회,"""방송장악은 군사독재로 회귀하는 것"" 한나라당이 추진 중인 언론법과 관련해 언론노조...","[(언론, 0.07052251496233784), (대구, 0.05593596010...",[운동],1
6,NIRW1900000001.9,사회,'만수'의 해에 국민들은 '명박'했다소의 해에는 뚜벅뚜벅 우보로 가자 지나간 한 해...,"[(서울, 0.06028149517063397), (시골, 0.05165174321...",[농가],1
7,NIRW1900000001.11,사회,"지리산 '2MB OUT 소망 띄우기', 경찰 저지로 무산 지리산 삼신봉에서 새해 첫...","[(국립공원, 0.085410966299339), (사무소, 0.0814812171...",[가방],1
8,NIRW1900000001.12,사회,"박희태 ""신라시대 만파식적은 어디 있는가?"" 청와대의 바람과 달리 소위 'MB법안'...","[(대표, 0.08654400270585133), (우리, 0.07221243989...",[],0
9,NIRW1900000001.17,기술,"모두에게 '윈-윈 게임'이 되는 세상, 아름다운 동행 21세기는 IQ(지능지수, I...","[(지수, 0.11923547124662498), (사회, 0.07665135440...",[],0


In [ ]:
match_list = []

for idx, row in tqdm(combined_df.iterrows()) :
    match_list.append(find_matches(unique_keys, row.keywords))
    
combined_df['match_keywords'] = match_list

2363373it [51:34, 1034.87it/s]

In [266]:
combined_df.iloc[190]

id                                               NIRW1900000001.341
topic                                                            사회
text              민선 첫 경기도교육감 선거 4월 8일 실시 경기도내 2백만 학생들의 교육을 책임질 ...
keywords          [(선거, 0.1029400006263013), (교육감, 0.08778827843...
match_keywords                                                 [광고]
match_count                                                       1
Name: 190, dtype: object

In [267]:
combined_df['match_count'] = combined_df['match_keywords'].apply(len)

In [357]:
combined_df

,id,topic,text,keywords,match_keywords,match_count
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643...","[배추, 채소, 수산물, 생선]",4
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231...",[],0
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402...",[운동],1
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118...",[],0
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118...","[어민, 양식, 어장, 생산량, 수확, 농약]",6
...,...,...,...,...,...,...
3611419,NZRW2200000001.2264,사회,日기자까지 통신조회 공수처에 아사히 “경위 밝히라” 일본 아사히신문이 30일 자사 ...,"[(기자, 0.07364789226727272), (조회, 0.05843554535...",[신문],1
3611420,NZRW2200000001.2265,사회,오늘 ‘노동자’로 인정받고 내일 쫓겨나는 방송작가 “이제 MBC ‘뉴스외전’ 작가는...,"[(작가, 0.05351025601963465), (근로, 0.05099118966...",[],0
3611421,NZRW2200000001.2266,사회,“내가 잣 도둑?” 앙심 품고 기자에 칼 겨눈 50대 집행유예 자신을 ‘잣 도둑’이...,"[(기자, 0.08305686810783024), (재판, 0.04992066025...",[식칼],1
3611422,NZRW2200000001.2269,사회,지상파 최초 KBS ‘비정규직 관리시스템’ 추진 KBS가 2023년 적용을 목표로 ...,"[(관리, 0.09326542432067701), (비정규직, 0.076670811...",[],0


In [358]:
get_keyword_only(combined_df.iloc[0].keywords)

['가락시장',
 '사람',
 '수산시장',
 '새벽',
 '시장',
 '배추',
 '서민',
 '경제',
 '상인',
 '가명',
 '채소',
 '도매',
 '수산물',
 '택시',
 '중간',
 '달라',
 '작년',
 '가격',
 '하루',
 '생선']

In [359]:
test_df = combined_df[:10]

In [360]:
test_df

,id,topic,text,keywords,match_keywords,match_count
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643...","[배추, 채소, 수산물, 생선]",4
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231...",[],0
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402...",[운동],1
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118...",[],0
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118...","[어민, 양식, 어장, 생산량, 수확, 농약]",6
5,NIRW1900000001.8,사회,"""방송장악은 군사독재로 회귀하는 것"" 한나라당이 추진 중인 언론법과 관련해 언론노조...","[(언론, 0.07052251496233784), (대구, 0.05593596010...",[운동],1
6,NIRW1900000001.9,사회,'만수'의 해에 국민들은 '명박'했다소의 해에는 뚜벅뚜벅 우보로 가자 지나간 한 해...,"[(서울, 0.06028149517063397), (시골, 0.05165174321...",[농가],1
7,NIRW1900000001.11,사회,"지리산 '2MB OUT 소망 띄우기', 경찰 저지로 무산 지리산 삼신봉에서 새해 첫...","[(국립공원, 0.085410966299339), (사무소, 0.0814812171...",[가방],1
8,NIRW1900000001.12,사회,"박희태 ""신라시대 만파식적은 어디 있는가?"" 청와대의 바람과 달리 소위 'MB법안'...","[(대표, 0.08654400270585133), (우리, 0.07221243989...",[],0
9,NIRW1900000001.17,기술,"모두에게 '윈-윈 게임'이 되는 세상, 아름다운 동행 21세기는 IQ(지능지수, I...","[(지수, 0.11923547124662498), (사회, 0.07665135440...",[현대],1


In [367]:
keyword_only = []
for index, row in combined_df.iterrows():
    k_only = get_keyword_only(row.keywords)
    keyword_only.append(k_only)
    
print(keyword_only[-1])
    

['서울시', '의결', '삭감', '예산안', '출연', '의회', '제시', '오세훈', '시장', '최종', '기준', '본회의', '기금', '가량', '대비', '금은', '예산', '복구', '운용', '관련']


In [368]:
combined_df['keyword_only'] = keyword_only

In [369]:
combined_df[:10]

,id,topic,text,keywords,match_keywords,match_count,keyword_only
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카...","[(가락시장, 0.03646997254429091), (사람, 0.034547643...","[배추, 채소, 수산물, 생선]",4,"[가락시장, 사람, 수산시장, 새벽, 시장, 배추, 서민, 경제, 상인, 가명, 채..."
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...","[(의원, 0.13532769889385685), (최소한, 0.0648119231...",[],0,"[의원, 최소한, 망언, 국민, 인간, 우리, 품위, 대한, 국회의원, 라면, 문제..."
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동...","[(시민, 0.06610684621307666), (경제, 0.06255996402...",[운동],1,"[시민, 경제, 일찍, 공원, 기념, 운동, 보상, 정치인, 대구, 국채, 행사, ..."
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...,"[(시민, 0.08943956506815211), (대전, 0.05810459118...",[],0,"[시민, 대전, 촛불, 이명박, 언론, 악법, 새해, 촛불집회, 노래, 노조, 거리..."
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...,"[(어민, 0.07520460843832412), (생산, 0.05677875118...","[어민, 양식, 어장, 생산량, 수확, 농약]",6,"[어민, 생산, 장흥, 양식, 김양, 무산, 잡태, 처음, 어장, 생산량, 소비자,..."
5,NIRW1900000001.8,사회,"""방송장악은 군사독재로 회귀하는 것"" 한나라당이 추진 중인 언론법과 관련해 언론노조...","[(언론, 0.07052251496233784), (대구, 0.05593596010...",[운동],1,"[언론, 대구, 방송, 재벌, 정부, 통해, 문화제, 파업, 정책, 촛불, 장악, ..."
6,NIRW1900000001.9,사회,'만수'의 해에 국민들은 '명박'했다소의 해에는 뚜벅뚜벅 우보로 가자 지나간 한 해...,"[(서울, 0.06028149517063397), (시골, 0.05165174321...",[농가],1,"[서울, 시골, 대학, 똥값, 골탑, 농가, 우보, 뚜벅뚜벅, 만수, 조금, 새해,..."
7,NIRW1900000001.11,사회,"지리산 '2MB OUT 소망 띄우기', 경찰 저지로 무산 지리산 삼신봉에서 새해 첫...","[(국립공원, 0.085410966299339), (사무소, 0.0814812171...",[가방],1,"[국립공원, 사무소, 통일, 행사, 지리산, 경찰, 띄우기, 자제, 새해, 요청, ..."
8,NIRW1900000001.12,사회,"박희태 ""신라시대 만파식적은 어디 있는가?"" 청와대의 바람과 달리 소위 'MB법안'...","[(대표, 0.08654400270585133), (우리, 0.07221243989...",[],0,"[대표, 우리, 한해, 돌밭, 맹세, 나라, 옥답, 박희태, 역사, 이란, 흥방, ..."
9,NIRW1900000001.17,기술,"모두에게 '윈-윈 게임'이 되는 세상, 아름다운 동행 21세기는 IQ(지능지수, I...","[(지수, 0.11923547124662498), (사회, 0.07665135440...",[현대],1,"[지수, 사회, 공존, 사람, 지능지수, 능력, 위해, 다른, 인맥, 모두, 성공,..."


In [370]:
sorted_df = combined_df.sort_values(by='match_count', ascending=False)

In [371]:
sorted_df[:10]

,id,topic,text,keywords,match_keywords,match_count,keyword_only
919285,NLRW1900000145.3232,사회,"치솟는 장바구니 물가…주부들 ""시장가기 무서워요"" 농축산물 가격 고공행진 28일 오...","[(닭고기, 0.05448024360688366), (가격, 0.0531123782...","[닭고기, 돼지고기, 수산물, 양파, 고구마, 과일, 채소, 축산물, 감자, 육류,...",12,"[닭고기, 가격, 시장, 돼지고기, 김씨, 물가, 대신, 수산물, 양파, 고구마, ..."
306059,NLRW1900000022.15219,사회,꽉찬 영양덩어리 ‘밤’ 별미로 즐기자 천고마비의 계절답게 각종 먹을거리가 눈길을 끌...,"[(양념장, 0.050500328376837794), (양파, 0.047964249...","[양파, 피망, 설탕, 후추, 닭꼬치, 소고기, 닭고기, 버터, 파이, 마늘, 참기...",12,"[양념장, 양파, 약간, 피망, 청주, 설탕, 소금, 불고기, 재료, 후추, 정도,..."
636297,NWRW1900000050.1782,사회,다이어트 도시락 배달주문해봤더니… 다이어트를 하면 ‘못 먹는 것’도 힘들지만 반대로...,"[(배달, 0.06682522208306246), (다이어트, 0.064935818...","[배달, 도시락, 브로콜리, 양배추, 냄비, 채소, 고구마, 냉동, 소시지, 냉장고...",12,"[배달, 다이어트, 주문, 도시락, 브로콜리, 정도, 데친, 양배추, 샐러드, 냄비..."
2662594,NWRW1900000002.17317,사회,홍천 찰옥수수 머핀 입맛 없는 요즘 간식으로 제격 7월 중순부터 수확하는 찰옥수수는...,"[(옥수수, 0.10049411681316021), (설탕, 0.0676543670...","[옥수수, 설탕, 버터, 베이컨, 요구르트, 생크림, 베이킹파우더, 파프리카, 우유...",12,"[옥수수, 설탕, 버터, 머핀, 크림, 베이컨, 소금, 가루, 달걀, 요구르트, 플..."
2673834,NWRW1900000003.8836,사회,"'新건강식' 마크로비오틱 열풍… 이젠 레스토랑 디저트까지 향신료 대신 막걸리, 초콜...","[(마크로비오틱, 0.0644685091709186), (대신, 0.05704785...","[버터, 케이크, 우유, 현미, 두유, 밀가루, 초콜릿, 채소, 토마토, 당근, 설탕]",11,"[마크로비오틱, 대신, 버터, 케이크, 재료, 디저트, 우유, 현미, 두유, 밀가루..."
2068449,NPRW2000000004.40572,사회,"추석 연휴, 남은 명절 음식 활용법 추석 연휴 남은 기간 남은 음식을 활용해 만들 ...","[(양파, 0.055806536554467434), (활용, 0.0553932090...","[양파, 참기름, 소고기, 과일, 간장, 파프리카, 설탕, 당근, 토마토, 사과, ...",11,"[양파, 활용, 음식, 준비, 표고버섯, 참기름, 소고기, 라이스, 과일, 간장, ..."
2746340,NWRW1900000007.9556,기술,'센서'가 '센스'를 갖추고 있다 스마트폰에 채소칸 보여주는 냉장고… 사람 행동 따...,"[(센서, 0.06591865660607013), (얼음, 0.04923969464...","[센서, 얼음, 냉장고, 채소, 에어컨, 신호, 세탁기, 가전, 스마트, 속옷, 사진]",11,"[센서, 얼음, 냉장고, 채소, 감지, 적외선, 에어컨, 신호, 주인, 밝기, 세탁..."
900063,NLRW1900000143.18348,경제,"대구 4분기 업종별경기 전반 회복세 전망…건설은 부진 제조업 전분기比 11P 상승,...","[(회복, 0.04750086728478365), (전망, 0.04210331110...","[전기, 기계, 종이, 음료, 인쇄, 의복, 고무, 화학, 광물, 섬유, 전자]",11,"[회복, 전망, 분기, 경기, 제조업, 업종, 부진, 상승, 비금속, 전기, 기계,..."
649991,NLRW1900000038.1247,사회,돈가스·불고기피자 이제 집에서 만들어요 외식으로는 먹지만 직접 집에서 해먹을 엄두는...,"[(양파, 0.03716004250301206), (돼지, 0.03586090766...","[양파, 감자, 치즈, 양배추, 고기, 피자, 고구마, 돈가스, 피망, 기름, 베이컨]",11,"[양파, 돼지, 감자, 치즈, 양배추, 불고기, 발사믹, 고기, 피자, 고구마, 돈..."
658971,NLRW1900000042.21,사회,웰빙시리즈(5·양배추 요리) 지난 월요일 서울 등 중부지방에는 100년 만에 폭설이...,"[(양배추, 0.07518286718311215), (볶음, 0.0362848234...","[양배추, 양파, 야채, 설탕, 당근, 고구마, 감자, 참기름, 마늘, 고기, 오이]",11,"[양배추, 볶음, 양파, 야채, 샐러드, 설탕, 당근, 고구마, 감자, 참기름, 마..."


In [375]:
def count_keyword_matches(keywords, ksic_key):
    result = {} # empty dict
    
    # 각 KSIC 산업에 대하여
    for ksic in ksic_key.items():
        intersect = keywords.intersection(ksic[1])
        if len(intersect) > 0 :
            result[ksic[0]] = intersect
            
    return result

In [379]:
result = count_keyword_matches(set(sorted_df.iloc[0]['keyword_only']), KSIC_key)

In [380]:
result

{'c101': {'닭고기', '돼지고기', '쇠고기', '육류', '축산물'},
 'c102': {'고등어', '수산물'},
 'c103': {'감자', '고구마', '과일', '양파', '채소'}}

In [381]:
test_df = sorted_df[:10]

In [382]:
test_df

,id,topic,text,keywords,match_keywords,match_count,keyword_only
919285,NLRW1900000145.3232,사회,"치솟는 장바구니 물가…주부들 ""시장가기 무서워요"" 농축산물 가격 고공행진 28일 오...","[(닭고기, 0.05448024360688366), (가격, 0.0531123782...","[닭고기, 돼지고기, 수산물, 양파, 고구마, 과일, 채소, 축산물, 감자, 육류,...",12,"[닭고기, 가격, 시장, 돼지고기, 김씨, 물가, 대신, 수산물, 양파, 고구마, ..."
306059,NLRW1900000022.15219,사회,꽉찬 영양덩어리 ‘밤’ 별미로 즐기자 천고마비의 계절답게 각종 먹을거리가 눈길을 끌...,"[(양념장, 0.050500328376837794), (양파, 0.047964249...","[양파, 피망, 설탕, 후추, 닭꼬치, 소고기, 닭고기, 버터, 파이, 마늘, 참기...",12,"[양념장, 양파, 약간, 피망, 청주, 설탕, 소금, 불고기, 재료, 후추, 정도,..."
636297,NWRW1900000050.1782,사회,다이어트 도시락 배달주문해봤더니… 다이어트를 하면 ‘못 먹는 것’도 힘들지만 반대로...,"[(배달, 0.06682522208306246), (다이어트, 0.064935818...","[배달, 도시락, 브로콜리, 양배추, 냄비, 채소, 고구마, 냉동, 소시지, 냉장고...",12,"[배달, 다이어트, 주문, 도시락, 브로콜리, 정도, 데친, 양배추, 샐러드, 냄비..."
2662594,NWRW1900000002.17317,사회,홍천 찰옥수수 머핀 입맛 없는 요즘 간식으로 제격 7월 중순부터 수확하는 찰옥수수는...,"[(옥수수, 0.10049411681316021), (설탕, 0.0676543670...","[옥수수, 설탕, 버터, 베이컨, 요구르트, 생크림, 베이킹파우더, 파프리카, 우유...",12,"[옥수수, 설탕, 버터, 머핀, 크림, 베이컨, 소금, 가루, 달걀, 요구르트, 플..."
2673834,NWRW1900000003.8836,사회,"'新건강식' 마크로비오틱 열풍… 이젠 레스토랑 디저트까지 향신료 대신 막걸리, 초콜...","[(마크로비오틱, 0.0644685091709186), (대신, 0.05704785...","[버터, 케이크, 우유, 현미, 두유, 밀가루, 초콜릿, 채소, 토마토, 당근, 설탕]",11,"[마크로비오틱, 대신, 버터, 케이크, 재료, 디저트, 우유, 현미, 두유, 밀가루..."
2068449,NPRW2000000004.40572,사회,"추석 연휴, 남은 명절 음식 활용법 추석 연휴 남은 기간 남은 음식을 활용해 만들 ...","[(양파, 0.055806536554467434), (활용, 0.0553932090...","[양파, 참기름, 소고기, 과일, 간장, 파프리카, 설탕, 당근, 토마토, 사과, ...",11,"[양파, 활용, 음식, 준비, 표고버섯, 참기름, 소고기, 라이스, 과일, 간장, ..."
2746340,NWRW1900000007.9556,기술,'센서'가 '센스'를 갖추고 있다 스마트폰에 채소칸 보여주는 냉장고… 사람 행동 따...,"[(센서, 0.06591865660607013), (얼음, 0.04923969464...","[센서, 얼음, 냉장고, 채소, 에어컨, 신호, 세탁기, 가전, 스마트, 속옷, 사진]",11,"[센서, 얼음, 냉장고, 채소, 감지, 적외선, 에어컨, 신호, 주인, 밝기, 세탁..."
900063,NLRW1900000143.18348,경제,"대구 4분기 업종별경기 전반 회복세 전망…건설은 부진 제조업 전분기比 11P 상승,...","[(회복, 0.04750086728478365), (전망, 0.04210331110...","[전기, 기계, 종이, 음료, 인쇄, 의복, 고무, 화학, 광물, 섬유, 전자]",11,"[회복, 전망, 분기, 경기, 제조업, 업종, 부진, 상승, 비금속, 전기, 기계,..."
649991,NLRW1900000038.1247,사회,돈가스·불고기피자 이제 집에서 만들어요 외식으로는 먹지만 직접 집에서 해먹을 엄두는...,"[(양파, 0.03716004250301206), (돼지, 0.03586090766...","[양파, 감자, 치즈, 양배추, 고기, 피자, 고구마, 돈가스, 피망, 기름, 베이컨]",11,"[양파, 돼지, 감자, 치즈, 양배추, 불고기, 발사믹, 고기, 피자, 고구마, 돈..."
658971,NLRW1900000042.21,사회,웰빙시리즈(5·양배추 요리) 지난 월요일 서울 등 중부지방에는 100년 만에 폭설이...,"[(양배추, 0.07518286718311215), (볶음, 0.0362848234...","[양배추, 양파, 야채, 설탕, 당근, 고구마, 감자, 참기름, 마늘, 고기, 오이]",11,"[양배추, 볶음, 양파, 야채, 샐러드, 설탕, 당근, 고구마, 감자, 참기름, 마..."


In [385]:
results = []
for index, row in tqdm(sorted_df.iterrows()):
    results.append(count_keyword_matches(set(row.keyword_only), KSIC_key))

3611424it [1:33:53, 641.06it/s] 


In [388]:
results[0]

{'c101': {'닭고기', '돼지고기', '쇠고기', '육류', '축산물'},
 'c102': {'고등어', '수산물'},
 'c103': {'감자', '고구마', '과일', '양파', '채소'}}

In [390]:
sorted_df['ksic_keys'] = results

In [391]:
sorted_df[:10]

,id,topic,text,keywords,match_keywords,match_count,keyword_only,ksic_keys
919285,NLRW1900000145.3232,사회,"치솟는 장바구니 물가…주부들 ""시장가기 무서워요"" 농축산물 가격 고공행진 28일 오...","[(닭고기, 0.05448024360688366), (가격, 0.0531123782...","[닭고기, 돼지고기, 수산물, 양파, 고구마, 과일, 채소, 축산물, 감자, 육류,...",12,"[닭고기, 가격, 시장, 돼지고기, 김씨, 물가, 대신, 수산물, 양파, 고구마, ...","{'c101': {'육류', '축산물', '쇠고기', '돼지고기', '닭고기'}, ..."
306059,NLRW1900000022.15219,사회,꽉찬 영양덩어리 ‘밤’ 별미로 즐기자 천고마비의 계절답게 각종 먹을거리가 눈길을 끌...,"[(양념장, 0.050500328376837794), (양파, 0.047964249...","[양파, 피망, 설탕, 후추, 닭꼬치, 소고기, 닭고기, 버터, 파이, 마늘, 참기...",12,"[양념장, 양파, 약간, 피망, 청주, 설탕, 소금, 불고기, 재료, 후추, 정도,...","{'c101': {'닭꼬치', '소고기', '닭고기'}, 'c103': {'피망',..."
636297,NWRW1900000050.1782,사회,다이어트 도시락 배달주문해봤더니… 다이어트를 하면 ‘못 먹는 것’도 힘들지만 반대로...,"[(배달, 0.06682522208306246), (다이어트, 0.064935818...","[배달, 도시락, 브로콜리, 양배추, 냄비, 채소, 고구마, 냉동, 소시지, 냉장고...",12,"[배달, 다이어트, 주문, 도시락, 브로콜리, 정도, 데친, 양배추, 샐러드, 냄비...","{'c101': {'냉동', '소시지'}, 'c103': {'채소', '브로콜리',..."
2662594,NWRW1900000002.17317,사회,홍천 찰옥수수 머핀 입맛 없는 요즘 간식으로 제격 7월 중순부터 수확하는 찰옥수수는...,"[(옥수수, 0.10049411681316021), (설탕, 0.0676543670...","[옥수수, 설탕, 버터, 베이컨, 요구르트, 생크림, 베이킹파우더, 파프리카, 우유...",12,"[옥수수, 설탕, 버터, 머핀, 크림, 베이컨, 소금, 가루, 달걀, 요구르트, 플...","{'c101': {'베이컨'}, 'c103': {'양파', '파프리카', '옥수수'..."
2673834,NWRW1900000003.8836,사회,"'新건강식' 마크로비오틱 열풍… 이젠 레스토랑 디저트까지 향신료 대신 막걸리, 초콜...","[(마크로비오틱, 0.0644685091709186), (대신, 0.05704785...","[버터, 케이크, 우유, 현미, 두유, 밀가루, 초콜릿, 채소, 토마토, 당근, 설탕]",11,"[마크로비오틱, 대신, 버터, 케이크, 재료, 디저트, 우유, 현미, 두유, 밀가루...","{'c103': {'채소', '당근', '토마토'}, 'c104': {'두유'}, ..."
2068449,NPRW2000000004.40572,사회,"추석 연휴, 남은 명절 음식 활용법 추석 연휴 남은 기간 남은 음식을 활용해 만들 ...","[(양파, 0.055806536554467434), (활용, 0.0553932090...","[양파, 참기름, 소고기, 과일, 간장, 파프리카, 설탕, 당근, 토마토, 사과, ...",11,"[양파, 활용, 음식, 준비, 표고버섯, 참기름, 소고기, 라이스, 과일, 간장, ...","{'c101': {'소고기'}, 'c103': {'사과', '파프리카', '토마토'..."
2746340,NWRW1900000007.9556,기술,'센서'가 '센스'를 갖추고 있다 스마트폰에 채소칸 보여주는 냉장고… 사람 행동 따...,"[(센서, 0.06591865660607013), (얼음, 0.04923969464...","[센서, 얼음, 냉장고, 채소, 에어컨, 신호, 세탁기, 가전, 스마트, 속옷, 사진]",11,"[센서, 얼음, 냉장고, 채소, 감지, 적외선, 에어컨, 신호, 주인, 밝기, 세탁...","{'c103': {'채소'}, 'c112': {'얼음'}, 'c141': {'속옷'..."
900063,NLRW1900000143.18348,경제,"대구 4분기 업종별경기 전반 회복세 전망…건설은 부진 제조업 전분기比 11P 상승,...","[(회복, 0.04750086728478365), (전망, 0.04210331110...","[전기, 기계, 종이, 음료, 인쇄, 의복, 고무, 화학, 광물, 섬유, 전자]",11,"[회복, 전망, 분기, 경기, 제조업, 업종, 부진, 상승, 비금속, 전기, 기계,...","{'c112': {'음료'}, 'c131': {'섬유'}, 'c134': {'섬유'..."
649991,NLRW1900000038.1247,사회,돈가스·불고기피자 이제 집에서 만들어요 외식으로는 먹지만 직접 집에서 해먹을 엄두는...,"[(양파, 0.03716004250301206), (돼지, 0.03586090766...","[양파, 감자, 치즈, 양배추, 고기, 피자, 고구마, 돈가스, 피망, 기름, 베이컨]",11,"[양파, 돼지, 감자, 치즈, 양배추, 불고기, 발사믹, 고기, 피자, 고구마, 돈...","{'c101': {'돈가스', '베이컨', '고기'}, 'c103': {'감자', ..."
658971,NLRW1900000042.21,사회,웰빙시리즈(5·양배추 요리) 지난 월요일 서울 등 중부지방에는 100년 만에 폭설이...,"[(양배추, 0.07518286718311215), (볶음, 0.0362848234...","[양배추, 양파, 야채, 설탕, 당근, 고구마, 감자, 참기름, 마늘, 고기, 오이]",11,"[양배추, 볶음, 양파, 야채, 샐러드, 설탕, 당근, 고구마, 감자, 참기름, 마...","{'c101': {'고기'}, 'c103': {'감자', '마늘', '오이', '양..."


In [397]:
sorted_df.iloc[23].ksic_keys

{'c191': {'석탄', '코크스', '타르'},
 'c201': {'탄소', '화학'},
 'c202': {'화학'},
 'c203': {'부산물'},
 'c204': {'화학'},
 'c239': {'타르', '탄소'},
 'c241': {'포스코'},
 'c259': {'자동차'},
 'c261': {'반도체'},
 'c282': {'배터리'},
 'c301': {'자동차'},
 'c303': {'자동차'}}

In [398]:
sorted_df.iloc[23].text

'포스코, 석탄 부산물로 고부가 탄소 소재 생산 포스코가 반도체와 자동차 배터리 소재로 사용되는 탄소 소재 ‘침상 코크스’ 생산에 나선다. 포스코는 22일 화학부문 계열사인 포스코컴텍과 일본 미쓰비시화학 등이 합작해 침상 코크스 생산 및 판매에 나설 예정이라고 밝혔다. 침상 코크스는 석탄을 고온처리할 때 발생하는 부산물인 콜타르에서 기름 성분을 제거한 뒤 열처리를 통해 만드는 고탄소 덩어리다. 반도체와 발광다이오드, 태양전지, 자동차 배터리 소재로 쓰인다. 포스코는 그동안 제철 과정에서 생기는 콜타르를 가공하지 않고 전량 판매해왔다. 그러나 이번 미쓰비시화학과의 합작을 통해 직접 침상 코크스를 만들어 팔 수 있게 됐다. 미쓰비시화학은 고급 침상 코크스를 만들 수 있는 세계에서 몇 안되는 업체로 꼽힌다. 침상 코크스를 제조·판매하게 되면 기존에 비해 5배가량 높은 수익을 올릴 수 있을 것으로 포스코는 전망했다. 포스코는 광양제철소 인근 22만6000㎡ 부지에 2014년까지 연산 10만t 규모의 침상 코크스 공장을 준공한다. 이 공장은 포스코컴텍이 지분의 60%를 투자하고, 미쓰비시화학과 미쓰비시상사가 각각 20%의 지분을 투자한다.'

In [399]:
sorted_df.iloc[23].keywords

[('코크스', 0.10139271401562122),
 ('침상', 0.10090641016143927),
 ('포스코', 0.08862348977224818),
 ('소재', 0.07187919122845124),
 ('미쓰비시', 0.06953934525366458),
 ('화학', 0.0637618549378745),
 ('생산', 0.06107862315561392),
 ('탄소', 0.04995913794395698),
 ('합작', 0.040248439664539976),
 ('자동차', 0.03983489953747543),
 ('배터리', 0.039669194930740725),
 ('반도체', 0.03605506908747186),
 ('컴텍', 0.03561164804565153),
 ('판매', 0.034827685525779015),
 ('공장', 0.03299373801066527),
 ('부산물', 0.03274603564115925),
 ('통해', 0.029518155952371977),
 ('석탄', 0.02705106089188819),
 ('투자', 0.026602528057215233),
 ('타르', 0.01770077818617181)]

In [401]:
KSIC_key

{'c101': ['도축장',
  '도축',
  '축산물',
  '도축업',
  '도축',
  '육류',
  '한우',
  '냉동',
  '토종닭',
  '축산',
  '마리',
  '사육',
  '고래고기',
  '햄',
  '소시지',
  '선지',
  '소고기',
  '돼지고기',
  '양고기',
  '오리고기',
  '닭고기',
  '훈제',
  '닭가슴살',
  '닭발',
  '닭갈비',
  '닭꼬치',
  '양꼬치',
  '돈까스',
  '돈가스',
  '족발',
  '갈비',
  '베이컨',
  '편육',
  '생산량',
  '쇠고기',
  '농민',
  '농가',
  '고기'],
 'c102': ['수산물',
  '양식',
  '수산',
  '광어',
  '항만',
  '해조',
  '어란',
  '어육',
  '캐비어',
  '어묵',
  '젓갈',
  '생선통조림',
  '생선페이스트',
  '건어물',
  '오징어',
  '우럭',
  '도미',
  '도다리',
  '가자미',
  '갈치',
  '고등어',
  '멍게',
  '해삼',
  '굴',
  '석화',
  '조개',
  '가리비',
  '키조개',
  '참돔',
  '돌돔',
  '줄돔',
  '방어',
  '쥐포',
  '먹태',
  '황태',
  '게장',
  '맛살',
  '생선가스',
  '생선까스',
  '액젓',
  '북어채',
  '황태채',
  '미역',
  '김부각',
  '구운김',
  '조미김',
  '생선',
  '김',
  '수확',
  '생산량',
  '어장',
  '어민',
  '아구',
  '아귀'],
 'c103': ['농업',
  '농가',
  '농산물',
  '농협',
  '식품',
  '양파',
  '마늘',
  '과일',
  '채소주스',
  '과일주스',
  '과채주스',
  '알로에',
  '과실주스',
  '딸기잼',
  '사과잼',
  '포도잼',
  '과일페이스트',
  '과일통조림',
  '과일샐러드',
  '김치',
  '단무지',

In [402]:
len(KSIC_key)

84

In [404]:
KSIC_key_count = KSIC_key

In [409]:
for key in KSIC_key_count.keys():
    KSIC_key_count[key] = 0

In [411]:
filtered_df = sorted_df[sorted_df['match_count'] > 1]

In [431]:
filtered_df.iloc[7].ksic_keys

{'c112': {'음료'},
 'c131': {'섬유'},
 'c134': {'섬유'},
 'c141': {'의복'},
 'c171': {'종이'},
 'c181': {'인쇄'},
 'c201': {'화학'},
 'c202': {'고무', '화학'},
 'c204': {'화학'},
 'c221': {'고무', '섬유'},
 'c239': {'섬유'},
 'c262': {'전기', '전자'},
 'c282': {'전기'},
 'c289': {'전기'},
 'c292': {'광물'},
 'c301': {'전기'},
 'c340': {'기계'}}

In [413]:
len(filtered_df)

694185

In [415]:
test_df = filtered_df[:10]

In [432]:
def find_max_keys(d):
    max_length = max(len(v) for v in d.values())
    max_keys = [k for k, v in d.items() if len(v) == max_length]
    return max_keys, max_length

In [433]:
find_max_keys(filtered_df.iloc[7].ksic_keys)

(['c202', 'c221', 'c262'], 2)

In [443]:
max_key_list = []
max_count_list = []

# 각 행에 대해 'max_key'와 'max_count' 계산 후 리스트에 추가
for dict_value in tqdm(filtered_df['ksic_keys']):
    max_keys, max_length = find_max_keys(dict_value)
    max_key_list.append(max_keys)
    max_count_list.append(max_length)

100%|█████████████████████████████████| 694185/694185 [08:40<00:00, 1332.97it/s]


In [444]:
filtered_df['max_keys'] = max_key_list

/tmp/ipykernel_6958/4135445346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['max_keys'] = max_key_list


In [445]:
filtered_df['max_count'] = max_count_list

/tmp/ipykernel_6958/2349844895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['max_count'] = max_count_list


In [464]:
save_df = filtered_df # save

In [465]:
df = filtered_df[filtered_df['max_count'] > 1]

In [468]:
for key in KSIC_key_count.keys():
    KSIC_key_count[key] = 0

for keys in tqdm(df['max_keys']):
    for key in keys:
        KSIC_key_count[key] += 1

100%|██████████████████████████████| 287292/287292 [00:00<00:00, 2064813.38it/s]


In [469]:
KSIC_key_count

{'c101': 13098,
 'c102': 5790,
 'c103': 36880,
 'c104': 68,
 'c105': 1289,
 'c106': 413,
 'c107': 3367,
 'c108': 402,
 'c111': 214,
 'c112': 2596,
 'c120': 101,
 'c131': 34,
 'c132': 130,
 'c133': 1,
 'c134': 246,
 'c139': 297,
 'c141': 163,
 'c142': 6,
 'c143': 13,
 'c144': 23,
 'c151': 439,
 'c152': 70,
 'c161': 22,
 'c162': 17,
 'c163': 0,
 'c171': 285,
 'c172': 2318,
 'c179': 362,
 'c181': 938,
 'c182': 770,
 'c191': 92,
 'c192': 15105,
 'c201': 5975,
 'c202': 1426,
 'c203': 723,
 'c204': 1152,
 'c211': 1903,
 'c212': 921,
 'c213': 441,
 'c221': 224,
 'c222': 706,
 'c231': 205,
 'c232': 102,
 'c233': 375,
 'c239': 133,
 'c241': 10976,
 'c242': 1846,
 'c243': 19,
 'c251': 686,
 'c252': 10666,
 'c259': 468,
 'c261': 7012,
 'c262': 16403,
 'c263': 1205,
 'c264': 26089,
 'c265': 326,
 'c266': 0,
 'c271': 43417,
 'c272': 7640,
 'c273': 5924,
 'c274': 178,
 'c281': 52,
 'c282': 2779,
 'c283': 740,
 'c284': 507,
 'c285': 4615,
 'c289': 63,
 'c291': 178,
 'c292': 142,
 'c301': 18516,
 'c30

In [472]:
sorted_KSIC_key_count = dict(sorted(KSIC_key_count.items(), key=lambda item: item[1], reverse=False))

In [475]:
sorted_KSIC_key_count

{'c163': 0,
 'c266': 0,
 'c304': 0,
 'c133': 1,
 'c142': 6,
 'c143': 13,
 'c162': 17,
 'c243': 19,
 'c161': 22,
 'c144': 23,
 'c131': 34,
 'c281': 52,
 'c289': 63,
 'c104': 68,
 'c152': 70,
 'c302': 81,
 'c191': 92,
 'c120': 101,
 'c232': 102,
 'c132': 130,
 'c239': 133,
 'c331': 133,
 'c292': 142,
 'c339': 158,
 'c141': 163,
 'c274': 178,
 'c291': 178,
 'c231': 205,
 'c111': 214,
 'c221': 224,
 'c134': 246,
 'c332': 252,
 'c171': 285,
 'c139': 297,
 'c265': 326,
 'c179': 362,
 'c233': 375,
 'c108': 402,
 'c312': 411,
 'c106': 413,
 'c320': 415,
 'c151': 439,
 'c213': 441,
 'c259': 468,
 'c284': 507,
 'c340': 684,
 'c251': 686,
 'c222': 706,
 'c203': 723,
 'c283': 740,
 'c333': 763,
 'c182': 770,
 'c212': 921,
 'c181': 938,
 'c204': 1152,
 'c263': 1205,
 'c105': 1289,
 'c202': 1426,
 'c242': 1846,
 'c211': 1903,
 'c172': 2318,
 'c112': 2596,
 'c319': 2625,
 'c334': 2648,
 'c282': 2779,
 'c107': 3367,
 'c285': 4615,
 'c102': 5790,
 'c273': 5924,
 'c201': 5975,
 'c261': 7012,
 'c272': 76

In [476]:
filtered_df['max_keys']

919285                 [c101, c103]
306059                       [c107]
636297                       [c103]
2662594                      [c105]
2673834                      [c107]
                     ...           
82870      [c201, c202, c204, c340]
479828                       [c271]
1223083                      [c271]
3461884                [c179, c273]
1669660                [c301, c311]
Name: max_keys, Length: 694185, dtype: object

In [456]:
def filter_rows_by_key(df, key_column, specific_key):
    """
    특정 key 값을 가지는 row를 필터링하여 새로운 DataFrame을 생성하는 함수

    Parameters:
    df (pd.DataFrame): 원본 DataFrame
    key_column (str): key 값이 저장된 열 이름
    specific_key (str): 필터링할 특정 key 값

    Returns:
    pd.DataFrame: 필터링된 새로운 DataFrame
    """
    new_df = df[df[key_column].apply(lambda keys: specific_key in keys)]
    
    return new_df

In [671]:
new_df = filter_rows_by_key(filtered_df, 'max_keys', 'c304')

In [672]:
new_df = new_df.sort_values(by='max_count', ascending=False)

In [673]:
new_df

,id,topic,text,keywords,match_keywords,match_count,keyword_only,ksic_keys,max_keys,max_count
175123,NIRW2100000001.5160,경제,"캐롯손보, '폰케어 액정안심보험' 출시…중고폰도 비대면 가입 캐롯손해보험이 국내 최...","[(보험, 0.10542997460748522), (액정, 0.05771829617...","[액정, 휴대폰, 케어, 수리, 영상, 중고]",6,"[보험, 액정, 가입, 휴대폰, 케어, 안심, 지능, 캐롯, 출시, 기술, 인공, ...","{'c262': {'액정'}, 'c264': {'휴대폰'}, 'c271': {'케어...","[c262, c264, c271, c272, c273, c304, c340]",1
1983972,NPRW1900000069.4677,기술,"""학부모와 함께 초중고 SW교육 의무화 대비 방안 찾는다"" 우리나라의 SW교육은 S...","[(한국, 0.08809809971860058), (교육, 0.08788215973...","[중고, 코딩]",2,"[한국, 교육, 정보통신기술사, 협회, 방안, 등록, 소프트웨어, 미래, 의무, 사...","{'c303': {'코딩'}, 'c304': {'중고'}}","[c303, c304]",1
923149,NLRW1900000145.10129,사회,"2사/07.07/경북경찰, 700억원대 환치기 26명 검거 경북지방경찰청은 7일 중...","[(베트남, 0.12679418669467749), (근로자, 0.099005360...","[중고, 자동차]",2,"[베트남, 근로자, 중고, 자동차, 불법, 경북, 외환거래, 국내, 경찰청, 대금,...","{'c259': {'자동차'}, 'c301': {'자동차'}, 'c303': {'자...","[c259, c301, c303, c304]",1
2249740,NPRW2100000007.10040,기술,"문체부 “OTT 징수율 낮은 건 사실, 음저협과 합의점 찾아야” 문화체육관광부가 온...","[(음악, 0.054344135114079595), (저작권료, 0.05367987...","[음악, 중고]",2,"[음악, 저작권료, 저작물, 이용, 문체부, 저작권, 서비스, 징수, 음저협, 통해...","{'c182': {'음악'}, 'c304': {'중고'}}","[c182, c304]",1
2880149,NWRW1900000014.12879,기술,‘재테크 게임’ 된 디아블로3 “부르는 대로 값을 쳐주겠다. 200만 원도 상관없으...,"[(한정판, 0.07529168155522077), (판매, 0.0668679675...","[게임, 중고]",2,"[한정판, 판매, 거래, 디아블로, 게임, 접속, 패키지, 사이트, 시작, 인기, ...","{'c304': {'중고'}, 'c334': {'게임'}}","[c304, c334]",1
...,...,...,...,...,...,...,...,...,...,...
1718561,NPRW1900000054.23338,경제,철망 제조머신 2억8천만원 중소기업진흥공단 유휴설비 포털사이트인'파인드 머신'(ww...,"[(기계, 0.09009584857132244), (매각, 0.06497349860...","[기계, 중고, 현대]",3,"[기계, 매각, 가격, 머신, 희망, 제작, 중고, 대표, 설비, 산업, 매물, 중...","{'c304': {'중고'}, 'c311': {'현대'}, 'c340': {'기계'}}","[c304, c311, c340]",1
1711431,NPRW1900000054.13170,경제,레이저 절단기 7500만원 중소기업진흥공단 유휴설비 포털사이트인 파인드머신(www....,"[(가공, 0.07546097739661704), (매각, 0.07471429800...","[기계, 레이저, 중고]",3,"[가공, 매각, 기계, 희망, 제작, 일본, 레이저, 중고, 매물, 주요, 세로, ...","{'c292': {'레이저'}, 'c304': {'중고'}, 'c340': {'기계'}}","[c292, c304, c340]",1
1711465,NPRW1900000054.13218,기술,"콧대꺾인 애플, 아이패드도 새제품으로 AS…한국 유일 새로 구매한 애플사의 아이패드...","[(제품, 0.08555968104057235), (기준, 0.06807789535...","[애플, 수리, 중고]",3,"[제품, 기준, 애플, 소비자, 교환, 국내, 공정위, 정책, 수리, 한국, 환불,...","{'c264': {'애플'}, 'c304': {'중고'}, 'c340': {'수리'}}","[c264, c304, c340]",1
2918096,NWRW1900000017.9004,사회,"비슷한 비용, 천양지차 급식… 영양사-밥쌤 정성이 갈랐다 탐사보도팀은 동아닷컴이 개...","[(학교, 0.0991788611735621), (급식, 0.086806866720...","[사진, 식품, 중고]",3,"[학교, 급식, 학생, 만족도, 때문, 영양사, 조사, 사진, 정성, 재료, 교사,...","{'c103': {'식품'}, 'c112': {'식품'}, 'c172': {'식품'...","[c103, c112, c172, c273, c304]",1


In [674]:
len(new_df)

1293

In [735]:
ksic = 'c304'
index = 200
print(new_df.iloc[index].ksic_keys)
print(new_df.iloc[index].keywords)
print(new_df.iloc[index].text)

{'c304': {'중고'}, 'c311': {'울산'}}
[('휴대전화', 0.06346381294009885), ('유씨', 0.04885687092642417), ('장물', 0.04744859648705614), ('매장', 0.04261191896223319), ('절도', 0.04184155218868987), ('도둑', 0.03402220325863033), ('경찰', 0.031973080849571484), ('울산', 0.03124387609725812), ('입건', 0.030713300732969767), ('중고', 0.030383876875844523), ('업자', 0.029406463033054338), ('안씨', 0.029380988128820947), ('처분', 0.028911114775058925), ('폭행', 0.027674256647137314), ('조직', 0.026977400004695246), ('보복', 0.024399580842202156), ('사실', 0.021549980998525283), ('혐의', 0.020264798574721216), ('남부', 0.01868066497931629), ('피해', 0.017999406148859417)]
‘법보다 주먹?’… 도둑 잡아 보복한 일당 철창행 매장 털리자 연합조직 가동… 도심 끌고 다니며 집단폭행 중고 휴대전화 매장서 장물 처분하다 덜미… 폭행ㆍ금품 빼앗아 경찰, 폭행 주범 2명 구속ㆍ7명 입건… 절도ㆍ장물처분 4명 입건 ''법보다 주먹이 가깝다''는 우스갯말이 있다. 하지만 현실에서 법보다 주먹을 먼저 휘두른 이들에게 베풀 관용은 없다. 휴대전화 매장에 든 도둑들을 직접 붙잡아 때리고 금품을 빼앗는 등 ''사적 보복''을 한 일당이 경찰에 붙잡혔다. 25일 울산 남부경찰서에 따르면 지난 12일 오전 3시께 경남 양산시 유모(34)씨의 중고 휴대전화 매장에 도둑이 들었다. 절도범 3명은 매장 창문으로 침입해 중고 휴대전화 23대(500만원 상당)를

In [486]:
new_df.iloc[0].ksic_keys

{'c103': {'김치'},
 'c111': {'와인'},
 'c134': {'의류'},
 'c143': {'의류'},
 'c163': {'코르크'},
 'c192': {'유지'},
 'c204': {'화장품'},
 'c272': {'온도'},
 'c285': {'냉장고'},
 'c311': {'건조'}}

In [345]:
filtered_df.loc[15312].keywords

[('지역', 0.10565919271315508),
 ('신문', 0.07440509659976717),
 ('조례', 0.0727950796346422),
 ('지원', 0.05336617843394162),
 ('인터넷', 0.042289370995583914),
 ('신문지', 0.03844810629627622),
 ('경남', 0.029174165432433813),
 ('발전', 0.028572290285860447),
 ('언론', 0.023251939436997996),
 ('대상', 0.022918388858204398),
 ('발의', 0.022781694473786338),
 ('전국', 0.020131558485716208),
 ('최초', 0.018359120428732865),
 ('관련', 0.018004859134064865),
 ('남도', 0.0165556103326489),
 ('도의', 0.01652421229314122),
 ('특별법', 0.01586242027037187),
 ('신문사', 0.015724656060741898),
 ('제정', 0.015637599942427),
 ('포함', 0.014262545253558027)]

In [346]:
filtered_df.loc[15312].text

'경남도, 전국 최초 지역신문지원조례 만든다 경상남도의회가 전국 최초로 지역신문지원조례를 만들었다. 이로써 중앙언론에 밀려 어려움을 겪고 있는 경남지역신문사들에 숨통이 트일 전망이다. 경남도의회는 16일 제281회 정례회 제4차 본회의에서 김해연(거제2, 진보), 문준희(합천, 한나라), 김윤근(통영1, 한나라) 도의원이 공동 발의한 \'경상남도 지역신문발전지원조례안\' 중 기획행정위원회가 수정한 조례안을 만장일치로 통과시켰다. 지역신문지원조례는 지역신문발전지원특별법을 근거해 만든 것으로, 경남도는 앞으로 2주 안에 관련 법률 검토를 거쳐 공포 여부를 결정할 예정이다. 이 조례가 공포되면 경상남도는 전국에서 최초로 지방조례를 통해 지역신문을 지원할 수 있게 된다. 경남도의회 발의로 통과한 이번 조례의 가장 큰 특징은 지역신문 지원 대상에 인터넷신문이 포함됐다는 점이다. 당초 발의안에는 지역신문발전지원특별법의 지역신문 정의에서 인터넷신문을 언급하지 않고 있음에 따라 인터넷신문은 지원 대상에서 빠져 있었다. 하지만 해당 상임위인 기획행정위가 지난 7일 이 조례안을 심의하는 과정에서 "인터넷신문 또한 지역신문의 역할을 충분히 하고 있다"는 판단 아래 지역신문의 정의에 인터넷신문을 포함시킴으로써 운명이 바뀌었다. 이로써 경남은 전국에서 최초로 지역신문지원조례를 만듦과 동시에 인터넷신문을 종이신문과 동등하게 대우하는 지자체가 될 전망이다. 이는 경남도와 유사한 지역신문지원조례 제정을 검토하고 있는 부산·충남·충북·전북 등에도 적잖은 영향을 미칠 전망이다. 이번 지역신문지원조례 만드는 일에 관여해온 경남민주언론시민연대 강창덕 대표는 "전국 최초라는 상징성도 있지만 지역차원에서 지역신문의 중요성과 또 이들이 겪고 있는 어려움을 인정했다는 게 크다"며 의미를 새겼다. 또 인터넷신문이 지원대상에 포함된 것과 관련해서는 "인터넷신문이란 새 매체의 속보성과 그 영향력을 종합적으로 판단한 결과"라며, "상위법에 언급돼 있지는 않으나 지원하지 말라는 근거 또한 없었다"고 말했다. 이

In [250]:
okt = Okt()

In [322]:
extract_keywords_tfidf(filtered_df.loc[11445].text)

,Word,TF-IDF Score
66,수주,0.319801
121,책임,0.248734
111,중공업,0.248734
135,한진,0.248734
29,노조,0.248734
10,고통,0.177667
56,분담,0.177667
143,회장,0.177667
6,경영,0.177667
25,노동자,0.177667


In [252]:
extract_keywords_textrank(filtered_df.loc[352].text)

['노동자',
 '조선',
 '대미',
 '용인',
 '기업',
 '의원',
 '사태',
 '정몽준',
 '울산',
 '해결',
 '회사',
 '농성',
 '정규직',
 '지난',
 '지역',
 '주인',
 '중공업',
 '대주',
 '굴뚝',
 '현대']